<a href="https://colab.research.google.com/github/GGpark1/User-Log-data-analytics/blob/main/refactoring_CF_computers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pandas.io import gbq

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import mean_squared_error
from scipy.sparse.linalg import svds
#pd.options.display.float_format = "{:,.2f}".format
import warnings
warnings.simplefilter("ignore")

## Collaborative Filtering 추천 시스템

### 추천 모델 구현을 위한 데이터 전처리
- Big-query를 활용하여 로그 데이터와 item 데이터 import
- 한 제품을 6개 이상 구매한 데이터 제외
- 구매 정보가 없는 데이터(Null)를 0으로 대체

In [3]:
sql = """SELECT user_id, product_id, count(*) AS purchase_count
FROM empirical-weft-350709.CP2_ggpark.log_data
WHERE category_code LIKE 'computers%' AND event_type = 'purchase'
GROUP BY user_id, product_id
HAVING purchase_count <= 5
ORDER BY purchase_count DESC"""
project_id = 'empirical-weft-350709'

df_limit = pd.read_gbq(sql, project_id=project_id, dialect='standard')

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725825577420-unm2gnkiprugilg743tkbig250f4sfsj.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=TKJTUOs8nDyoAEY12cK1qpsDYNqzX2&prompt=consent&access_type=offline
Enter the authorization code: 4/1AX4XfWgOcnL56zvDz7-CGIyIhqvx1auNapGXEAYmRqpe5yBY1w31LttoZ7Q


In [4]:
sql = """SELECT product_id, brand, category_code FROM `empirical-weft-350709.CP2_ggpark.log_data`
WHERE event_type = 'purchase' AND category_code LIKE 'computers%'
GROUP BY product_id, brand, category_code"""
project_id = 'empirical-weft-350709'

df_product = pd.read_gbq(sql, project_id=project_id, dialect='standard')

In [5]:
df_limit.sort_values('purchase_count')

,user_id,product_id,purchase_count
10647,554397121,25900015,1
14201,533030009,1480466,1
14200,528999142,1480680,1
14199,539220656,1480300,1
14198,545971281,1480233,1
...,...,...,...
74,516135809,1306747,5
75,556423632,1307345,5
76,554223289,1307074,5
78,553393549,1307361,5


In [6]:
df_ratings_limit = df_limit
mean = df_ratings_limit['purchase_count'].mean()

In [7]:
df_fin_limit = pd.pivot_table(df_limit, index = ['user_id'], columns = ['product_id'], values = 'purchase_count')
df_fin_limit

product_id,1303049,1303355,1304106,1304297,1304346,1304392,1304409,1304410,1304411,1304438,...,45500013,45500014,45500015,45500019,45500021,45500022,45500027,45500028,45500029,45500030
user_id,,,,,,,,,,,,,,,,,,,,,
418115333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
420935067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
445228646,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
465723791,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
479752015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566196738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566201318,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566204832,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df_final_limit = df_fin_limit.fillna(0)

In [9]:
df_base_line_limit = df_fin_limit.fillna(mean)

In [10]:
df_final_limit['total'] = df_final_limit.sum(axis=1)
df_filtered_limit = df_final_limit.loc[df_final_limit['total'] > 1]
df_filtered_limit.drop(columns='total', inplace = True)

df_filtered_limit

product_id,1303049,1303355,1304106,1304297,1304346,1304392,1304409,1304410,1304411,1304438,...,45500013,45500014,45500015,45500019,45500021,45500022,45500027,45500028,45500029,45500030
user_id,,,,,,,,,,,,,,,,,,,,,
445228646,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
479752015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
512363815,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
512364184,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
512364323,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565970301,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
566011852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
566015450,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Collaborative Filtering 구현
- SVD 행렬 분해 방식으로 구현
    - 차원 수가 많은 데이터에 유리한 알고리즘
    - 희소 문제에 유리한 알고리즘

In [11]:
# Matrix 계산을 위해 pandas를 numpy로 변환
matrix_limit = df_filtered_limit.values

# 유저의 평균 구매횟수
user_ratings_mean_limit = np.mean(matrix_limit, axis = 1)

# 각 유저의 구매횟수에서 평균 구매횟수를 뺌으로써 평균에서 벗어난 특이값을 남김
user_ratings_mean_reshape_limit = user_ratings_mean_limit.reshape(-1, 1)
matrix_user_mean_limit = matrix_limit - user_ratings_mean_reshape_limit

In [12]:
# 행렬 분해
U_limit, sigma_limit, Vt_limit = svds(matrix_user_mean_limit, k = 1000)

In [13]:
# 0이 포함된 대칭행렬로 전환
sigma_limit = np.diag(sigma_limit)

In [14]:
# 분해한 행렬에서 특이값을 추출한 후 다시 하나로 복구함
dot_matrix_limit = np.dot(U_limit, sigma_limit)
df_svd_predicted_purchase_limit = np.dot(dot_matrix_limit, Vt_limit)

# 앞서 뺐던 유저의 평균 구매 횟수를 다시 더함
df_svd_predicted_purchase_limit = df_svd_predicted_purchase_limit + user_ratings_mean_reshape_limit

In [15]:
# 유저별로 제품에 대한 구매 확률을 예측한 테이블

df_svd_preds_limit = pd.DataFrame(df_svd_predicted_purchase_limit, columns = df_filtered_limit.columns, index = df_filtered_limit.index)
df_svd_preds_limit.head()

product_id,1303049,1303355,1304106,1304297,1304346,1304392,1304409,1304410,1304411,1304438,...,45500013,45500014,45500015,45500019,45500021,45500022,45500027,45500028,45500029,45500030
user_id,,,,,,,,,,,,,,,,,,,,,
445228646,-0.000003,0.000002,-0.000003,0.000003,-0.000003,0.000017,-0.000005,0.000013,0.000129,-0.000012,...,-0.000003,-0.000002,0.000009,0.000002,0.000016,0.000013,-0.000012,-0.000003,0.000018,-0.000006
479752015,-0.000039,-0.000006,-0.000039,0.000002,-0.000039,-0.000241,0.000090,-0.000051,-0.001977,0.000379,...,-0.000039,0.000046,0.000017,0.000023,-0.000020,-0.000014,-0.000030,-0.000039,0.000085,0.000214
512363815,0.000065,-0.000078,0.000065,-0.000338,0.000065,-0.000468,1.001212,0.005131,-0.006477,-0.000633,...,0.000065,0.000308,-0.000348,-0.005479,-0.002027,-0.000127,-0.000223,0.000065,-0.017498,-0.006246
512364184,0.000002,-0.000007,0.000002,0.000005,0.000002,-0.001009,0.000029,-0.000015,0.000303,-0.000047,...,0.000002,-0.000034,0.000002,0.000060,-0.000052,-0.000047,0.000007,0.000002,-0.000081,0.000135
512364323,0.000128,0.000121,0.000128,0.000085,0.000128,0.000097,0.000455,-0.000295,0.119358,0.000181,...,0.000128,-0.000041,0.000017,0.002323,-0.000622,-0.000020,0.000859,0.000128,0.000186,-0.016059


In [16]:
# 유저 Id를 입력하면 추천 아이템이 출력되는 함수

def recommend_items(df_svd_preds, user_id, ori_items_df, ori_ratings_df, num_recommendations=5):
    
    user_row_number = user_id 
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 데이터 정렬 -> 구매 확률이 높은 순으로 정렬 됌
    sorted_user_predictions = df_svd_preds.loc[user_row_number].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_ratings_df[ori_ratings_df.user_id == user_id]
    
    # 위에서 뽑은 user_data와 원본 item 데이터를 합친다. 
    user_history = user_data.merge(ori_items_df, on = 'product_id').sort_values(['purchase_count'], ascending=False)
    
    # 유저가 이미 구매한 item을 제외하고 데이터 추출
    recommendations = ori_items_df[~ori_items_df['product_id'].isin(user_history['product_id'])]

    # 정렬된 추천 list와 item 정보를 merge하여 하나의 테이블로 만듦 
    recommendations = recommendations.merge(pd.DataFrame(sorted_user_predictions).reset_index(), on = 'product_id')
    recommendations = recommendations.rename(columns = {user_row_number: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
                      

    return user_history, recommendations

In [18]:
already_rated, predictions = recommend_items(df_svd_preds_limit, 512363815, df_product, df_ratings_limit, 10)

In [19]:
predictions

,product_id,brand,category_code,Predictions
1586,1306571,acer,computers.notebook,0.142998
994,1306832,lenovo,computers.notebook,0.086788
2610,17800075,None,computers.desktop,0.069365
1407,1701515,lg,computers.peripherals.monitor,0.069365
768,11600095,defender,computers.desktop,0.058644
1865,11600186,defender,computers.desktop,0.038518
1609,1701009,samsung,computers.peripherals.monitor,0.034265
1505,6600944,zeppelin,computers.components.memory,0.031786
1269,6801092,samsung,computers.components.hdd,0.022437
1280,9200673,ritmix,computers.peripherals.keyboard,0.020554


### 평가지표
- RMSE : 실제 데이터의 잠재 요인을 얼마나 잘 추출했는지 원본 데이터와의 오차로 비교함
- Base Line
    - 평균 구매 횟수로 채운 테이블과 원본 데이터와의 MSE를 산출하여 모든 사용자가 평균만큼은 구매할 것으로 예측하고 오차를 계산함
    - Score : 0.85
- Truncated SVD 
    - SVD로 예측된 테이블과 원본 데이터와의 MSE를 산출하여 추출된 잠재 요인만큼 상품을 구매할 것으로 예측하고 오차를 계산함
    - Score : 0.22


In [20]:
def evaluate(sparse_matrix, pred_matrix):
        idx, jdx = sparse_matrix.to_numpy().nonzero()
        ys, preds = [], []
        for i, j in zip(idx, jdx):
            ys.append(sparse_matrix.iloc[i, j])
            preds.append(pred_matrix.iloc[i, j])

        error = mean_squared_error(ys, preds)
        return np.sqrt(error)

In [21]:
# Base Line -> 예측 데이터를 평균으로 채워 넣음
# Mean_squared_error
# 구매 횟수에 필터를 걸지 않은 결과

evaluate(df_filtered_limit, df_base_line_limit)

0.8508147409811612

In [22]:
# 행렬 분해 방식으로 도출한 예측 데이터
# Mean_squared_error
# 구매 횟수에 필터를 걸지 않은 결과

evaluate(df_filtered_limit, df_svd_preds_limit)

0.22831300945010938